# Train a model on RAF DB 

## Test DataLoaders

In [ ]:


import torch 
import numpy as np
from generator_RAF import RAFDataset
import os
import torchvision as tv
import torch
import matplotlib.pyplot as plt

size=(160,120)
data_transforms = tv.transforms.Compose([
        tv.transforms.Resize(size),
        tv.transforms.ToTensor(),
    ])

train_dataset = RAFDataset(
        d_set='train',
        params=[False],
        transform=data_transforms
    )
print(len(train_dataset))
img,target=next(iter(train_dataset))
plt.imshow(img.permute(1,2,0))

# Get Data

In [ ]:
from get_raf_data import *
size=(160,120)
batch_size=16
trainloader,validloader,testloader=load_data(batch_size=batch_size,size=size,data_augmentation=False)

## Load Model

In [ ]:
from get_model import load_model

model,optimizer,criterion,device=load_model(baseline='inception_resnetV1',freeze_layer='block8',
          GPU=0,lr=1e-5)




In [ ]:
#initialize global variables
filename=os.path.join('./Checkpoints/','Inception_resnetV1_RafDB_block8'+str(size)\
                       +'_'+str(batch_size)+'.pth.tar')

csv_path=os.path.join('./Logs/','Inception_resnetV1_RafDB_block8'+str(size)\
                       +'_'+str(batch_size)+'.csv')

# Train Model

In [ ]:
from train_model_raf import train_model
from torch.utils.tensorboard import SummaryWriter
tb = SummaryWriter()

train_model(tb,model,optimizer,criterion,
          trainloader,validloader, filename, csv_path,
          device=device)

tb.close()

# Grid Search

In [1]:
from itertools import product
from global_main import main
import os
parameters = dict(
    lr = [1e-5, 1e-4],
    batch_size = [16],
    layer= ['block8'],
    dropout_prob=[0.6,0.3,0.0],
    data_augmentation = [False]
)
model_name= 'inception_resnetV1'
param_values = [v for v in parameters.values()]
print(param_values)
print(f'num combinations={len(list(product(*param_values)))}')
for lr,batch_size, layer, dropout_prob, data_augmentation in product(*param_values):
    filename = '_'.join((model_name, str(batch_size), str(layer), str(lr), str(dropout_prob)))
    print(filename)
    params={'model_name':model_name,
            'filename': filename,
            'lr': lr,
            'batch_size': batch_size,
            'layer': layer, 
            'dropout_prob': dropout_prob,
            'data_augmentation': data_augmentation}
    print(model_name, lr, batch_size, layer, dropout_prob, data_augmentation)
    
    main(**params)

[[1e-05, 0.0001], [16], ['block8'], [0.6, 0.3, 0.0], [False]]
num combinations=6
./runs/inception_resnetV1_16_block8_1e-05_0.6
inception_resnetV1 1e-05 16 block8 0.6 False
training parameters: 
model_name = inception_resnetV1
filename = ./runs/inception_resnetV1_16_block8_1e-05_0.6
lr = 1e-05
batch_size = 16
layer = block8
dropout_prob = 0.6
data_augmentation = False
num batch train: 691
num batch validation: 77
num batch test: 192


/Users/tom/anaconda3/envs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[1 2 3 4 5 6 7], y=[5 4 6 ... 7 7 7] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Train
Epoch: 1/20
691/691 [==================================================] - 665s 962ms/step - loss: 1.8222 - acc: 0.2578
=> Saving a new best


FileNotFoundError: [Errno 2] No such file or directory: './Checkpoints/./runs/inception_resnetV1_16_block8_1e-05_0.6.pth.tar'